### Imports

In [1]:
import numpy as np
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
import cvxpy as cp 

Question  d)

In [2]:
from scipy.optimize import minimize, NonlinearConstraint, Bounds, LinearConstraint
from numpy import cos, pi

In [3]:
# constants
n = 10
alpha = 1.5
rmin = 1.0
rmax = 2.0
theta = 2 * pi / (5 * (n + 1))
costh = cos(theta)  # radians

In [55]:
### Linear constraints ###
lin_const = []

# rmin <= r_i <= rmax (2)
A2 = LinearConstraint(np.eye(n), rmin, rmax)
lin_const.append(A2)

# -theta*alpha <= r_{i+1} - r_i <= theta*alpha for i = 1, ..., n (4)
M = (np.eye(n+1) - np.eye(n+1, k=-1))[1:-1, :-1]

# col 1 is r1, col 2 is r2, etc.
A4 = LinearConstraint(M, -theta*alpha, theta*alpha)
lin_const.append(A4)

# i = 0
Mr1 = np.append(1, np.zeros(n-1))
Ar1 = LinearConstraint(Mr1, rmin-theta*alpha, rmin+theta*alpha)
lin_const.append(Ar1)

# i = n
Mrn = np.append(np.zeros(n-1), -1)
Arn = LinearConstraint(Mrn, -rmax-theta*alpha, -rmax+theta*alpha)
lin_const.append(Arn)

In [56]:
### Nonlinear constraints ###
# 2r_{i-1}*r_{i+1}*cos(theta) - r_i*r_{i-1} - r_i*r_{i+1} <= 0 for i = 0, ..., n+1 (3)
non_lin_const = []

# i = 0
nlcon_0 = NonlinearConstraint(lambda r: 2*rmin*r[0]*costh - rmin**2 - rmin*r[0], -np.inf, 0)
non_lin_const.append(nlcon_0)

# i = 1
nlcon_1 = NonlinearConstraint(lambda r: 2*rmin*r[1]*costh - r[0]*rmin - r[0]*r[1], -np.inf, 0)
non_lin_const.append(nlcon_1)

# i = 2...n-1
for i in range(2, n):
    non_lin_const.append(NonlinearConstraint(lambda r: 2*r[i-2]*r[i]*costh - r[i-1]*r[i-2] - r[i-1]*r[i], -np.inf, 0))

# i = n
nlcon_n = NonlinearConstraint(lambda r: 2*r[n-2]*rmax*costh - r[n-1]*r[n-2] - r[n-1]*rmax, -np.inf, 0)
non_lin_const.append(nlcon_n)

# i = n+1
nlcon_np1 = NonlinearConstraint(lambda r: 2*r[n-1]**2 * costh - rmax*r[n-1] - rmax*r[n-1], -np.inf, 0)
non_lin_const.append(nlcon_np1)

In [57]:
### Objective function ###
# maximize sum of r_i
def obj(r):
    return -np.mean(r)

In [58]:
# initial design (r0)
r0 = rmin*np.ones(n)

### Solve ###
res = minimize(obj, r0,
    # method="trust-constr",
    constraints=[*lin_const, *non_lin_const]
)

# print results
print(res)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.582670692507787
       x: [ 1.013e+00  1.041e+00  1.212e+00  1.383e+00  1.555e+00
            1.726e+00  1.897e+00  2.000e+00  2.000e+00  2.000e+00]
     nit: 6
     jac: [-1.000e-01 -1.000e-01 -1.000e-01 -1.000e-01 -1.000e-01
           -1.000e-01 -1.000e-01 -1.000e-01 -1.000e-01 -1.000e-01]
    nfev: 66
    njev: 6


In [59]:
print(len(non_lin_const) + len(lin_const))

16
